In [1]:
import re
all_query = """
%sql
WITH address_cte AS (
  SELECT id, ARRAY_AGG(
    STRUCT(
      ad.address_type,
      ad.address_line_1,
      ad.address_line_2,
      ad.city,
      ad.state,
      CASE
        WHEN LENGTH(ad.zipcode) = 10 THEN SUBSTRING(ad.zipcode, 1, 5)
        WHEN LENGTH(ad.zipcode) = 5 THEN zipcode
        ELSE NULL
      END AS zip_code,
      CASE
        WHEN LENGTH(ad.zipcode) = 10 THEN SUBSTRING(ad.zipcode, 7)
        WHEN LENGTH(ad.zipcode) = 4 THEN ad.zipcode
        ELSE NULL
      END AS postal_code,
      'USA' AS country
    )
  ) AS addressinfo
  FROM address_temp_table ad
  GROUP BY id
  HAVING COUNT(id) >= 1
),
contact_cte AS (
    SELECT ct.id, ARRAY_AGG(STRUCT(ct.usage_type, ct.phone)) AS contact_info
    FROM contact_temp_table ct
    GROUP BY ct.id
    HAVING COUNT(id) >= 1
),
email_validation_cte AS (
  SELECT d.id, CASE
    WHEN d.email NOT LIKE '%@%.%' THEN 'NULL'
    ELSE d.email
    END AS email
  FROM details_temp_table d
),
national_id_validation_cte AS (
   SELECT d.id, CASE
      WHEN LENGTH(REPLACE(d.ssn, '-', '')) > 9 OR LENGTH(REPLACE(d.ssn, '-', '')) < 9 THEN NULL
      ELSE CONCAT(SUBSTR(d.ssn, 1, 3), '-', SUBSTR(REPLACE(d.ssn, '-', ''), 4, 2), '-', SUBSTR(REPLACE(d.ssn, '-', ''), 6))
    END AS formatted_ssn
  FROM details_temp_table d
),
date_validation_cte AS (
  SELECT id,
  CASE
    WHEN to_date(replace(d.date_of_birth, '/', '-'), 'M-d-yyyy') > current_date() THEN current_date()
    ELSE to_date(replace(d.date_of_birth, '/', '-'), 'M-d-yyyy')
  END AS date_of_birth,
  CASE
    WHEN to_date(replace(d.date_of_birth, '/', '-'), 'M-d-yyyy') > current_date() then year(current_date())
    ELSE year(to_date(replace(d.date_of_birth, '/', '-'), 'M-d-yyyy'))
  END AS year_of_birth,
  CASE
    when d.deceased_date is not null then true
    else false
  END AS deceased_ind,
  CASE
      WHEN d.deceased_date > current_date() THEN year(current_date()) - year(to_date(replace(d.date_of_birth, '/', '-'), 'M-d-yyyy'))
      ELSE year(to_date(replace(d.deceased_date, '/', '-'), 'M-d-yyyy')) - year(to_date(replace(d.date_of_birth, '/', '-'), 'M-d-yyyy'))
    END AS deceased_age,
  CASE
      WHEN d.deceased_date > current_date() THEN NULL
      ELSE (to_date(replace(d.deceased_date, '/', '-'), 'M-d-yyyy'))
    END AS deceased_date
FROM details_temp_table d
),
language_validation_cte AS (
  SELECT id, CASE
    WHEN d.spoken_language_1 = d.spoken_language_2 THEN ARRAY(d.spoken_language_1)
    ELSE ARRAY(d.spoken_language_1, d.spoken_language_2)
  END AS languages
FROM details_temp_table d
)
INSERT INTO junee_target_table
SELECT
  h.id AS source_id,
  h.insurer_id AS subscriber_id,
  d.first_name AS first_name,
  d.middle_name  AS middle_name,
  d.last_name AS last_name,
  CASE
    WHEN d.gender = 'M' THEN 'Mr.'
    WHEN d.gender = 'F' AND d.marital_status IN ('Single', 'Divorced', NULL) THEN 'Ms.'
    WHEN d.gender = 'F' AND d.marital_status IN ('Married', 'Widowed') THEN 'Mrs.'
    ELSE 'Unknown'
  END AS prefix_name,
  NULL AS suffix_name,
  CASE
    WHEN d.middle_name IS NULL THEN CONCAT(d.first_name, ' ', d.last_name)
    ELSE CONCAT(d.first_name, ' ', d.middle_name, ' ', d.last_name)
  END AS full_name,
  'Nova' AS record_source,
  CURRENT_TIMESTAMP() AS record_timestamp,
  FALSE AS is_verified,
  a.addressinfo AS address,
  c.contact_info AS phone,
  e.email AS email,
  FALSE AS privacy_preference,
  n.formatted_ssn AS national_id,
  d.gender AS gender,
  d.marital_status AS marital_status,
  dv.date_of_birth AS date_of_birth,
  dv.year_of_birth AS year_of_birth,
  dv.deceased_ind AS deceased_indicator,
  dv.deceased_age AS deceased_age,
  dv.deceased_date AS deceased_date,
  l.languages AS languages,
  STRUCT(d.company, d.job_role,
    CASE
      WHEN d.deceased_date IS NULL THEN 'Active'
      ELSE 'Inactive'
    END AS status,
    CASE
      WHEN d.date_of_birth > d.job_hiredate THEN NULL
      WHEN d.job_hiredate > d.deceased_date THEN NULL
      ELSE TO_DATE(REPLACE(d.job_hiredate, '/', '-'), 'M-d-yyyy')
    END AS hire_date) AS employment/*employment*/,
  map('relation with subscriber', h.relationship) as additional_source_value
FROM header_temp_table h
INNER JOIN details_temp_table d ON h.id = d.id
LEFT JOIN address_cte a ON h.id = a.id

LEFT JOIN contact_cte c ON h.id = c.id
LEFT JOIN email_validation_cte e ON h.id = e.id
LEFT JOIN national_id_validation_cte n ON h.id = n.id
LEFT JOIN date_validation_cte dv ON h.id = dv.id
LEFT JOIN language_validation_cte l ON h.id = l.id
WHERE NOT EXISTS (
  SELECT 1
  FROM junee_target_table
  WHERE junee_target_table.source_id = h.id
);
"""
insert_query_pattern = r"INSERT\s+INTO[^;]+;"

insert_query = re.search(insert_query_pattern, all_query, re.IGNORECASE)
if insert_query:
    insert_query = insert_query.group()
print(insert_query)


INSERT INTO junee_target_table
SELECT
  h.id AS source_id,
  h.insurer_id AS subscriber_id,
  d.first_name AS first_name,
  d.middle_name  AS middle_name,
  d.last_name AS last_name,
  CASE
    WHEN d.gender = 'M' THEN 'Mr.'
    WHEN d.gender = 'F' AND d.marital_status IN ('Single', 'Divorced', NULL) THEN 'Ms.'
    WHEN d.gender = 'F' AND d.marital_status IN ('Married', 'Widowed') THEN 'Mrs.'
    ELSE 'Unknown'
  END AS prefix_name,
  NULL AS suffix_name,
  CASE
    WHEN d.middle_name IS NULL THEN CONCAT(d.first_name, ' ', d.last_name)
    ELSE CONCAT(d.first_name, ' ', d.middle_name, ' ', d.last_name)
  END AS full_name,
  'Nova' AS record_source,
  CURRENT_TIMESTAMP() AS record_timestamp,
  FALSE AS is_verified,
  a.addressinfo AS address,
  c.contact_info AS phone,
  e.email AS email,
  FALSE AS privacy_preference,
  n.formatted_ssn AS national_id,
  d.gender AS gender,
  d.marital_status AS marital_status,
  dv.date_of_birth AS date_of_birth,
  dv.year_of_birth AS year_of_birth,
  

In [2]:
def remove_strings_by_index(strings, indices):
    indices.sort(reverse=True)  # Sort the indices in descending order

    for index in indices:
        if 0 <= index < len(strings):
            strings.pop(index)

    return strings


In [3]:
def extract_words(string):
    column_names = []
    aliases = []
    start_index = string.lower().find('select')
    # print(start_index)
    # exit();
    if start_index == -1:
        return column_names, aliases

    substring = string[start_index + 6:]  # Skip 'SELECT' or 'select' (6 characters)

    parts = substring[:substring.lower().find('from')].split(',')
    # print(substring)
    # exit();
    norm = parts
    l=[]
    exc_ind=[]
    hold=0
    b_count=0
    disp=""
    indexx=[]
    ind2=[]
    tri=1
    cc=0
    for x,i in enumerate(parts):

      if  i.count('(')!=0 or hold == 1:

        if tri==1:
          ind2.append(x-cc)

          cc=cc-1


        cc=cc+1


        disp=disp+","+i
        hold = 1
        b_count = b_count+ i.count('(')
        indexx.append(x)

        b_count = b_count- i.count(')')
        tri=0
        if b_count==0 and hold == 1:
          tri=1

          l.append(disp)
          disp=""
          hold = 0




    # print(ind2)
    result = remove_strings_by_index(norm,indexx)
    # final = result+l
    # print(indexx)
    # print(ind2)
    for c,data in enumerate(l):
      result.insert(ind2[c], data)


    dic = {}
    dic2 = {}
    l1=[]
    l2=[]
    # print(result)
    for part in result:
      dic[(part[1:part.lower().rfind(' as ') ])]=(part[part.lower().rfind(' as ') + len(' as '):].strip())
      dic2[(part[part.lower().rfind(' as ') + len(' as '):].strip())]=(part[1:part.lower().rfind(' as ') ])
      l1.append(part[1:part.lower().rfind(' as ') ])
      l2.append(part[part.lower().rfind(' as ') + len(' as '):].strip())



    return dic2



In [4]:
dict_mapping_attribute=extract_words(insert_query)


In [5]:
dict_mapping_attribute

{'source_id': '  h.id',
 'subscriber_id': '  h.insurer_id',
 'first_name': '  d.first_name',
 'middle_name': '  d.middle_name ',
 'last_name': '  d.last_name',
 'prefix_name': "\n  CASE\n    WHEN d.gender = 'M' THEN 'Mr.'\n    WHEN d.gender = 'F' AND d.marital_status IN ('Single', 'Divorced', NULL) THEN 'Ms.'\n    WHEN d.gender = 'F' AND d.marital_status IN ('Married', 'Widowed') THEN 'Mrs.'\n    ELSE 'Unknown'\n  END",
 'suffix_name': '  NULL',
 'full_name': "\n  CASE\n    WHEN d.middle_name IS NULL THEN CONCAT(d.first_name, ' ', d.last_name)\n    ELSE CONCAT(d.first_name, ' ', d.middle_name, ' ', d.last_name)\n  END",
 'record_source': "  'Nova'",
 'record_timestamp': '\n  CURRENT_TIMESTAMP()',
 'is_verified': '  FALSE',
 'address': '  a.addressinfo',
 'phone': '  c.contact_info',
 'email': '  e.email',
 'privacy_preference': '  FALSE',
 'national_id': '  n.formatted_ssn',
 'gender': '  d.gender',
 'marital_status': '  d.marital_status',
 'date_of_birth': '  dv.date_of_birth',
 'year

In [6]:
transformations = list(dict_mapping_attribute.values())
print(transformations)

['  h.id', '  h.insurer_id', '  d.first_name', '  d.middle_name ', '  d.last_name', "\n  CASE\n    WHEN d.gender = 'M' THEN 'Mr.'\n    WHEN d.gender = 'F' AND d.marital_status IN ('Single', 'Divorced', NULL) THEN 'Ms.'\n    WHEN d.gender = 'F' AND d.marital_status IN ('Married', 'Widowed') THEN 'Mrs.'\n    ELSE 'Unknown'\n  END", '  NULL', "\n  CASE\n    WHEN d.middle_name IS NULL THEN CONCAT(d.first_name, ' ', d.last_name)\n    ELSE CONCAT(d.first_name, ' ', d.middle_name, ' ', d.last_name)\n  END", "  'Nova'", '\n  CURRENT_TIMESTAMP()', '  FALSE', '  a.addressinfo', '  c.contact_info', '  e.email', '  FALSE', '  n.formatted_ssn', '  d.gender', '  d.marital_status', '  dv.date_of_birth', '  dv.year_of_birth', '  dv.deceased_ind', '  dv.deceased_age', '  dv.deceased_date', '  l.languages', "\n  STRUCT(d.company, d.job_role,\n    CASE\n      WHEN d.deceased_date IS NULL THEN 'Active'\n      ELSE 'Inactive'\n    END AS status,\n    CASE\n      WHEN d.date_of_birth > d.job_hiredate THEN N

In [7]:
transformations_key = list(dict_mapping_attribute.keys())
print(transformations_key)

['source_id', 'subscriber_id', 'first_name', 'middle_name', 'last_name', 'prefix_name', 'suffix_name', 'full_name', 'record_source', 'record_timestamp', 'is_verified', 'address', 'phone', 'email', 'privacy_preference', 'national_id', 'gender', 'marital_status', 'date_of_birth', 'year_of_birth', 'deceased_indicator', 'deceased_age', 'deceased_date', 'languages', 'employment/*employment*/', 'additional_source_value']


In [ ]:
trans_lists = []

for transformation in transformations:
    # Remove leading/trailing spaces
    trans_lists.append(transformation.strip())
print(trans_lists)


['h.id', 'h.insurer_id', 'd.first_name', 'd.middle_name', 'd.last_name', "CASE\n    WHEN d.gender = 'M' THEN 'Mr.'\n    WHEN d.gender = 'F' AND d.marital_status IN ('Single', 'Divorced', NULL) THEN 'Ms.'\n    WHEN d.gender = 'F' AND d.marital_status IN ('Married', 'Widowed') THEN 'Mrs.'\n    ELSE 'Unknown'\n  END", 'NULL', "CASE\n    WHEN d.middle_name IS NULL THEN CONCAT(d.first_name, ' ', d.last_name)\n    ELSE CONCAT(d.first_name, ' ', d.middle_name, ' ', d.last_name)\n  END", "'Nova'", 'CURRENT_TIMESTAMP()', 'FALSE', 'a.addressinfo', 'c.contact_info', 'e.email', 'FALSE', 'n.formatted_ssn', 'd.gender', 'd.marital_status', 'dv.date_of_birth', 'dv.year_of_birth', 'dv.deceased_ind', 'dv.deceased_age', 'dv.deceased_date', 'l.languages', "STRUCT(d.company, d.job_role,\n    CASE\n      WHEN d.deceased_date IS NULL THEN 'Active'\n      ELSE 'Inactive'\n    END AS status,\n    CASE\n      WHEN d.date_of_birth > d.job_hiredate THEN NULL\n      WHEN d.job_hiredate > d.deceased_date THEN NULL\

In [ ]:
def main_table_name(all_query):
    # Regular expression pattern to match table names after FROM or JOIN,
    # excluding CTE definitions
    pattern = r'FROM\s+(\w+)\s+'
    excluded_tables = set(re.findall(r'INSERT\s+INTO\s+(\w+)\s+', all_query, re.IGNORECASE))


    # Find all matches of the pattern in the SQL script
    matches = re.findall(pattern, all_query, re.IGNORECASE)

    # Remove duplicates while preserving the order
    unique_table_names = list(dict.fromkeys(matches))
    filtered_table_names = [table_name for table_name in unique_table_names if table_name not in excluded_tables]

    return filtered_table_names

tables_after_from = main_table_name(all_query)

print(tables_after_from)

['address_temp_table', 'contact_temp_table', 'details_temp_table', 'header_temp_table']


In [ ]:
#get all the tables(temp/perm) and its alias
import re

def extract_table_name(all_query):
# Regular expression pattern to match table names after FROM or JOIN,
    # excluding CTE definitions
    pattern = r'(?:FROM|JOIN)\s+(\w+)\s+'
    excluded_tables = set(re.findall(r'INSERT\s+INTO\s+(\w+)\s+', all_query, re.IGNORECASE))

    # Find all matches of the pattern in the SQL script
    matches = re.findall(pattern, all_query, re.IGNORECASE)

    # Remove duplicates while preserving the order
    unique_table_names = list(dict.fromkeys(matches))

    filtered_table_names = [table_name for table_name in unique_table_names if table_name not in excluded_tables]

    return filtered_table_names



def extract_table_aliases(all_query):
    pattern = r'(?:FROM|JOIN)\s+\w+\s+(?:AS\s+)?(\w+)\s+'

    matches = re.findall(pattern, all_query, re.IGNORECASE)

    unique_table_aliases = list(dict.fromkeys(matches))

    return unique_table_aliases

tables_after_from = extract_table_name(all_query)
aliases = extract_table_aliases(all_query)

table_alias_dict = {}

for table, alias in zip(tables_after_from, aliases):
    table_alias_dict[table] = alias

print(table_alias_dict)

{'address_temp_table': 'ad', 'contact_temp_table': 'ct', 'details_temp_table': 'd', 'header_temp_table': 'h', 'address_cte': 'a', 'contact_cte': 'c', 'email_validation_cte': 'e', 'national_id_validation_cte': 'n', 'date_validation_cte': 'dv', 'language_validation_cte': 'l'}


In [ ]:
#all the tables
all_tables = list(table_alias_dict.keys())
print(all_tables)

['address_temp_table', 'contact_temp_table', 'details_temp_table', 'header_temp_table', 'address_cte', 'contact_cte', 'email_validation_cte', 'national_id_validation_cte', 'date_validation_cte', 'language_validation_cte']


In [ ]:
regular_tables = main_table_name(all_query)
cte_tables = [item for item in all_tables if item not in regular_tables]

print(cte_tables)

['address_cte', 'contact_cte', 'email_validation_cte', 'national_id_validation_cte', 'date_validation_cte', 'language_validation_cte']


In [ ]:
matches = []

for trans_list in trans_lists:
    match_group = re.findall(r'\b((?:h\.)?\w+\.\w+)\b', trans_list)
    if match_group:
        if len(match_group) == 1:
            matches.append(match_group[0])
        else:
            matches.append(tuple(match_group))
    else:
        matches.append(None)

print(matches)


['h.id', 'h.insurer_id', 'd.first_name', 'd.middle_name', 'd.last_name', ('d.gender', 'd.gender', 'd.marital_status', 'd.gender', 'd.marital_status'), None, ('d.middle_name', 'd.first_name', 'd.last_name', 'd.first_name', 'd.middle_name', 'd.last_name'), None, None, None, 'a.addressinfo', 'c.contact_info', 'e.email', None, 'n.formatted_ssn', 'd.gender', 'd.marital_status', 'dv.date_of_birth', 'dv.year_of_birth', 'dv.deceased_ind', 'dv.deceased_age', 'dv.deceased_date', 'l.languages', ('d.company', 'd.job_role', 'd.deceased_date', 'd.date_of_birth', 'd.job_hiredate', 'd.job_hiredate', 'd.deceased_date', 'd.job_hiredate'), 'h.relationship']


In [ ]:
separated_list = []

for item in matches:
    if item is None:
        separated_list.append({"table": None, "column": None})
    elif isinstance(item, tuple):
        column_objects = []
        added_items = set()
        for sub_item in item:
            if sub_item is not None and '.' in sub_item:
                table, column = sub_item.split('.')
                if (table, column) not in added_items:
                    column_objects.append({"table": table, "column": column})
                    added_items.add((table, column))
        separated_list.append(column_objects)
    else:
        table, column = item.split('.')
        separated_list.append({"table": table, "column": column})

separated_list


[{'table': 'h', 'column': 'id'},
 {'table': 'h', 'column': 'insurer_id'},
 {'table': 'd', 'column': 'first_name'},
 {'table': 'd', 'column': 'middle_name'},
 {'table': 'd', 'column': 'last_name'},
 [{'table': 'd', 'column': 'gender'},
  {'table': 'd', 'column': 'marital_status'}],
 {'table': None, 'column': None},
 [{'table': 'd', 'column': 'middle_name'},
  {'table': 'd', 'column': 'first_name'},
  {'table': 'd', 'column': 'last_name'}],
 {'table': None, 'column': None},
 {'table': None, 'column': None},
 {'table': None, 'column': None},
 {'table': 'a', 'column': 'addressinfo'},
 {'table': 'c', 'column': 'contact_info'},
 {'table': 'e', 'column': 'email'},
 {'table': None, 'column': None},
 {'table': 'n', 'column': 'formatted_ssn'},
 {'table': 'd', 'column': 'gender'},
 {'table': 'd', 'column': 'marital_status'},
 {'table': 'dv', 'column': 'date_of_birth'},
 {'table': 'dv', 'column': 'year_of_birth'},
 {'table': 'dv', 'column': 'deceased_ind'},
 {'table': 'dv', 'column': 'deceased_age

In [ ]:
def get_table_from_alias(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    return None  # Return None if value is not found in the dictionary


# Function usage example to check
value_to_find = 'a'
get_table_name = get_table_from_alias(table_alias_dict, value_to_find)
print(get_table_name)
  # Output: address_cte


address_cte


In [ ]:
# replace alias with table name that include cte too
replace_alias_with_list = []

for item in matches:
    if item is None:
        replace_alias_with_list.append({"table": None, "column": None})
    elif isinstance(item, tuple):
        column_objects = []
        added_items = set()
        for sub_item in item:
            if sub_item is not None and '.' in sub_item:
                table, column = sub_item.split('.')
                if (table, column) not in added_items:
                    column_objects.append({"table": get_table_from_alias(table_alias_dict, table), "column": column})
                    added_items.add((table, column))
        replace_alias_with_list.append(column_objects)
    else:
        table, column = item.split('.')
        replace_alias_with_list.append({"table": get_table_from_alias(table_alias_dict, table), "column": column})

replace_alias_with_list


[{'table': 'header_temp_table', 'column': 'id'},
 {'table': 'header_temp_table', 'column': 'insurer_id'},
 {'table': 'details_temp_table', 'column': 'first_name'},
 {'table': 'details_temp_table', 'column': 'middle_name'},
 {'table': 'details_temp_table', 'column': 'last_name'},
 [{'table': 'details_temp_table', 'column': 'gender'},
  {'table': 'details_temp_table', 'column': 'marital_status'}],
 {'table': None, 'column': None},
 [{'table': 'details_temp_table', 'column': 'middle_name'},
  {'table': 'details_temp_table', 'column': 'first_name'},
  {'table': 'details_temp_table', 'column': 'last_name'}],
 {'table': None, 'column': None},
 {'table': None, 'column': None},
 {'table': None, 'column': None},
 {'table': 'address_cte', 'column': 'addressinfo'},
 {'table': 'contact_cte', 'column': 'contact_info'},
 {'table': 'email_validation_cte', 'column': 'email'},
 {'table': None, 'column': None},
 {'table': 'national_id_validation_cte', 'column': 'formatted_ssn'},
 {'table': 'details_temp

In [ ]:
import re

def find_dependent_table(cte_name, cte_query):
    regex = r'\b' + cte_name + r'\s+AS\s+\(([^;]+)\)'
    match = re.search(regex, cte_query, re.IGNORECASE)

    if match:
        subquery = match.group(1)
        subquery_regex = r'\bFROM\s+([a-zA-Z0-9_]+)\b|\bJOIN\s+([a-zA-Z0-9_]+)\b'
        subquery_match = re.search(subquery_regex, subquery, re.IGNORECASE)

        if subquery_match:
            return subquery_match.group(1) or subquery_match.group(2)

    return None


In [ ]:
cte_name = 'date_validation_cte'
dependent_table = find_dependent_table(cte_name, all_query)
print(dependent_table)


details_temp_table


In [ ]:
# Replace cte with table by checking depentent table
source_table = []
grouped_tables = []
for item in replace_alias_with_list:
    if isinstance(item, dict) and 'table' in item:
        table_name = item['table']
        if table_name in cte_tables:
            dependent_table = find_dependent_table(table_name, all_query)
            if isinstance(dependent_table, list):
                grouped_tables.extend([{"table": dependent_table, "column": item['column']}])
            else:
                grouped_tables.append({"table": dependent_table, "column": item['column']})
        else:
            grouped_tables.append(item)
    elif isinstance(item, list):
        grouped_subtables = []
        for sub_item in item:
            table_name = sub_item['table']
            if table_name in cte_tables:
                print(f"{table_name} is a CTE table")
                dependent_table = find_dependent_table(table_name, all_query)
                if isinstance(dependent_table, list):
                    grouped_subtables.extend([{"table": dependent_table, "column": sub_item['column']}])
                else:
                    grouped_subtables.append({"table": dependent_table, "column": sub_item['column']})
            else:
                grouped_subtables.append(sub_item)
        grouped_tables.append(grouped_subtables)

source_table.extend(grouped_tables)
source_table


[{'table': 'header_temp_table', 'column': 'id'},
 {'table': 'header_temp_table', 'column': 'insurer_id'},
 {'table': 'details_temp_table', 'column': 'first_name'},
 {'table': 'details_temp_table', 'column': 'middle_name'},
 {'table': 'details_temp_table', 'column': 'last_name'},
 [{'table': 'details_temp_table', 'column': 'gender'},
  {'table': 'details_temp_table', 'column': 'marital_status'}],
 {'table': None, 'column': None},
 [{'table': 'details_temp_table', 'column': 'middle_name'},
  {'table': 'details_temp_table', 'column': 'first_name'},
  {'table': 'details_temp_table', 'column': 'last_name'}],
 {'table': None, 'column': None},
 {'table': None, 'column': None},
 {'table': None, 'column': None},
 {'table': 'address_temp_table', 'column': 'addressinfo'},
 {'table': 'contact_temp_table', 'column': 'contact_info'},
 {'table': 'details_temp_table', 'column': 'email'},
 {'table': None, 'column': None},
 {'table': 'details_temp_table', 'column': 'formatted_ssn'},
 {'table': 'details_

In [ ]:
# merged the data that are in loop
data = replace_alias_with_list

table_column_dicts = []

for item in data:
    if isinstance(item, dict):
        table_column_dicts.append(item)
    elif isinstance(item, list):
        table = item[0]['table']
        columns = ', '.join([x['column'] for x in item])
        merged_data = {'table': table, 'column': columns}
        table_column_dicts.append(merged_data)

table_column_dicts


[{'table': 'header_temp_table', 'column': 'id'},
 {'table': 'header_temp_table', 'column': 'insurer_id'},
 {'table': 'details_temp_table', 'column': 'first_name'},
 {'table': 'details_temp_table', 'column': 'middle_name'},
 {'table': 'details_temp_table', 'column': 'last_name'},
 {'table': 'details_temp_table', 'column': 'gender, marital_status'},
 {'table': None, 'column': None},
 {'table': 'details_temp_table',
  'column': 'middle_name, first_name, last_name'},
 {'table': None, 'column': None},
 {'table': None, 'column': None},
 {'table': None, 'column': None},
 {'table': 'address_cte', 'column': 'addressinfo'},
 {'table': 'contact_cte', 'column': 'contact_info'},
 {'table': 'email_validation_cte', 'column': 'email'},
 {'table': None, 'column': None},
 {'table': 'national_id_validation_cte', 'column': 'formatted_ssn'},
 {'table': 'details_temp_table', 'column': 'gender'},
 {'table': 'details_temp_table', 'column': 'marital_status'},
 {'table': 'date_validation_cte', 'column': 'date_o

In [ ]:
tables = []
columns = []

for item in table_column_dicts:
    table = item['table']
    column = item['column']

    if table is not None and column is not None:
        tables.append(table)
        columns.append(column)
    else:
        tables.append(table)
        columns.append(column)

print("Tables:", tables)
print("Columns:", columns)


Tables: ['header_temp_table', 'header_temp_table', 'details_temp_table', 'details_temp_table', 'details_temp_table', 'details_temp_table', None, 'details_temp_table', None, None, None, 'address_cte', 'contact_cte', 'email_validation_cte', None, 'national_id_validation_cte', 'details_temp_table', 'details_temp_table', 'date_validation_cte', 'date_validation_cte', 'date_validation_cte', 'date_validation_cte', 'date_validation_cte', 'language_validation_cte', 'details_temp_table', 'header_temp_table']
Columns: ['id', 'insurer_id', 'first_name', 'middle_name', 'last_name', 'gender, marital_status', None, 'middle_name, first_name, last_name', None, None, None, 'addressinfo', 'contact_info', 'email', None, 'formatted_ssn', 'gender', 'marital_status', 'date_of_birth', 'year_of_birth', 'deceased_ind', 'deceased_age', 'deceased_date', 'languages', 'company, job_role, deceased_date, date_of_birth, job_hiredate', 'relationship']


In [ ]:
count = len(columns)
print("Count:", count)

Count: 26


In [ ]:
result = [{'table': tables, 'column': columns, 'trans': trans_lists} for tables, columns, trans_lists in zip(tables, columns, trans_lists)]
result

[{'table': 'header_temp_table', 'column': 'id', 'trans': 'h.id'},
 {'table': 'header_temp_table',
  'column': 'insurer_id',
  'trans': 'h.insurer_id'},
 {'table': 'details_temp_table',
  'column': 'first_name',
  'trans': 'd.first_name'},
 {'table': 'details_temp_table',
  'column': 'middle_name',
  'trans': 'd.middle_name'},
 {'table': 'details_temp_table',
  'column': 'last_name',
  'trans': 'd.last_name'},
 {'table': 'details_temp_table',
  'column': 'gender, marital_status',
  'trans': "CASE\n    WHEN d.gender = 'M' THEN 'Mr.'\n    WHEN d.gender = 'F' AND d.marital_status IN ('Single', 'Divorced', NULL) THEN 'Ms.'\n    WHEN d.gender = 'F' AND d.marital_status IN ('Married', 'Widowed') THEN 'Mrs.'\n    ELSE 'Unknown'\n  END"},
 {'table': None, 'column': None, 'trans': 'NULL'},
 {'table': 'details_temp_table',
  'column': 'middle_name, first_name, last_name',
  'trans': "CASE\n    WHEN d.middle_name IS NULL THEN CONCAT(d.first_name, ' ', d.last_name)\n    ELSE CONCAT(d.first_name, ' 

In [ ]:
def extract_cte_words(string):
    column_names = []
    aliases = []
    # start_index = string.lower()

    # if start_index == -1:
    #     return column_names, aliases

    substring = string

    # end_index = substring.lower()
    parts = substring.split(',')
    # print(parts)

    norm = parts
    l=[]
    exc_ind=[]
    hold=0
    b_count=0
    disp=""
    indexx=[]
    ind2=[]
    tri=1
    cc=0
    for x,i in enumerate(parts):
        if i.count('(')!=0 or hold == 1:
            if tri==1:
                ind2.append(x-cc)
                cc=cc-1


            cc=cc+1

            disp=disp+","+i
            hold = 1
            b_count = b_count+ i.count('(')
            indexx.append(x)

            b_count = b_count- i.count(')')
            tri=0
            if b_count==0 and hold == 1:
                tri=1

                l.append(disp)
                disp=""
                hold = 0

    result = remove_strings_by_index(norm, indexx)

    for c,data in enumerate(l):
        result.insert(ind2[c], data)

    dic = {}
    l1=[]
    l2=[]

    for part in result:
        dic[(part[1:part.lower().rfind(' as ') ])]=(part[part.lower().rfind(' as ') + len(' as '):].strip())
        l1.append(part[1:part.lower().rfind(' as ') ])
        l2.append(part[part.lower().rfind(' as ') + len(' as '):].strip())

    return dic




In [ ]:
def split_select(query_string):
    query_string = query_string.lower()
    select_keyword = "select"
    from_keyword = "from"
    result = []

    while select_keyword in query_string and from_keyword in query_string:
        select_index = query_string.index(select_keyword)
        from_index = query_string.index(from_keyword)
        selected_columns = query_string[select_index + len(select_keyword):from_index].strip()
        result.append(selected_columns)
        query_string = query_string[from_index + len(from_keyword):]

    return result

In [ ]:
data = split_select(all_query)

In [ ]:
def get_cte_query(cte_alias):
  for i in data:
      if i.find(f"as {cte_alias},") == -1:
          thing = extract_cte_words(i)
          # print(thing)
          for z in range(len(thing)):
              if list(thing.values())[z].find(cte_alias) != -1:
                  return(list(thing.keys())[z])

      if i.find(f"as {cte_alias},") != -1:
          thing = extract_cte_words(i)
          for z in range(len(thing)):
              if list(thing.values())[z].find(cte_alias) != -1:
                  return(list(thing.keys())[z])

In [ ]:
get_cte_query('addressinfo')

" array_agg(\n    struct(\n      ad.address_type,\n      ad.address_line_1,\n      ad.address_line_2,\n      ad.city,\n      ad.state,\n      case\n        when length(ad.zipcode) = 10 then substring(ad.zipcode, 1, 5)\n        when length(ad.zipcode) = 5 then zipcode\n        else null\n      end as zip_code,\n      case\n        when length(ad.zipcode) = 10 then substring(ad.zipcode, 7)\n        when length(ad.zipcode) = 4 then ad.zipcode\n        else null\n      end as postal_code,\n      'usa' as country\n    )\n  )"

In [ ]:
import re

def find_matches(term):
    matches = re.findall(r'\b((?:h\.)?\w+\.\w+)\b', term)

    if matches:
        column_names = ','.join(set(match.split('.')[1] for match in matches))
        return column_names



In [ ]:
find_matches("array_agg(\n    struct(\n      ad.address_type,\n      ad.address_line_1,\n      ad.address_line_2,\n      ad.city,\n      ad.state,\n      case\n        when length(ad.zipcode) = 10 then substring(ad.zipcode, 1, 5)\n        when length(ad.zipcode) = 5 then zipcode\n        else null\n      end as zip_code,\n      case\n        when length(ad.zipcode) = 10 then substring(ad.zipcode, 7)\n        when length(ad.zipcode) = 4 then ad.zipcode\n        else null\n      end as postal_code,\n      'usa' as country\n    )\n  )"
)

'address_type,address_line_1,address_line_2,state,zipcode,city'

In [ ]:
get_cte_queries = get_cte_query('addressinfo')
print(get_cte_queries)

 array_agg(
    struct(
      ad.address_type,
      ad.address_line_1,
      ad.address_line_2,
      ad.city,
      ad.state,
      case
        when length(ad.zipcode) = 10 then substring(ad.zipcode, 1, 5)
        when length(ad.zipcode) = 5 then zipcode
        else null
      end as zip_code,
      case
        when length(ad.zipcode) = 10 then substring(ad.zipcode, 7)
        when length(ad.zipcode) = 4 then ad.zipcode
        else null
      end as postal_code,
      'usa' as country
    )
  )


In [ ]:
cte_query_table = []
cte_query = []
for item in result:
    if isinstance(item, dict) and 'table' in item:
        table_name = item['table']
        column_name = item['column']
        if table_name in cte_tables:
            dependent_table = find_dependent_table(table_name, all_query)
            get_cte_queries = get_cte_query(column_name)
            if isinstance(dependent_table, list):
                cte_query.extend([{"table": dependent_table, "column":find_matches(get_cte_queries), "trans": get_cte_queries }])
            else:
                cte_query.append({"table": dependent_table, "column":find_matches(get_cte_queries), "trans": get_cte_queries})
        else:
            cte_query.append(item)
    elif isinstance(item, list):
        grouped_subtables = []
        for sub_item in item:
            table_name = sub_item['table']
            if table_name in cte_tables:
                print(f"{table_name} is a CTE table")
                dependent_table = find_dependent_table(table_name, all_query)
                get_cte_query = get_cte_query(column_name)

                if isinstance(dependent_table, list):
                    grouped_subtables.extend([{"table": dependent_table, "column":find_matches(get_cte_queries), "trans": get_cte_queries}])
                else:
                    grouped_subtables.append({"table": dependent_table, "column":find_matches(get_cte_queries), "trans": get_cte_queries})
            else:
                grouped_subtables.append(sub_item)
        cte_query.append(grouped_subtables)

cte_query_table.extend(cte_query)
cte_query_table


[{'table': 'header_temp_table', 'column': 'id', 'trans': 'h.id'},
 {'table': 'header_temp_table',
  'column': 'insurer_id',
  'trans': 'h.insurer_id'},
 {'table': 'details_temp_table',
  'column': 'first_name',
  'trans': 'd.first_name'},
 {'table': 'details_temp_table',
  'column': 'middle_name',
  'trans': 'd.middle_name'},
 {'table': 'details_temp_table',
  'column': 'last_name',
  'trans': 'd.last_name'},
 {'table': 'details_temp_table',
  'column': 'gender, marital_status',
  'trans': "CASE\n    WHEN d.gender = 'M' THEN 'Mr.'\n    WHEN d.gender = 'F' AND d.marital_status IN ('Single', 'Divorced', NULL) THEN 'Ms.'\n    WHEN d.gender = 'F' AND d.marital_status IN ('Married', 'Widowed') THEN 'Mrs.'\n    ELSE 'Unknown'\n  END"},
 {'table': None, 'column': None, 'trans': 'NULL'},
 {'table': 'details_temp_table',
  'column': 'middle_name, first_name, last_name',
  'trans': "CASE\n    WHEN d.middle_name IS NULL THEN CONCAT(d.first_name, ' ', d.last_name)\n    ELSE CONCAT(d.first_name, ' 

In [ ]:
tables = []
columns = []
transformations = []

for item in cte_query_table:
    table = item['table']
    column = item['column']
    trans = item['trans']
    if table is not None and column is not None and trans is not None:
        tables.append(table)
        columns.append(column)
        transformations.append(trans)
    else:
        tables.append(table)
        columns.append(column)
        transformations.append(trans)

print("Tables:", tables)
print("Columns:", columns)
print("Transformations:", transformations)


Tables: ['header_temp_table', 'header_temp_table', 'details_temp_table', 'details_temp_table', 'details_temp_table', 'details_temp_table', None, 'details_temp_table', None, None, None, 'address_temp_table', 'contact_temp_table', 'details_temp_table', None, 'details_temp_table', 'details_temp_table', 'details_temp_table', 'details_temp_table', 'details_temp_table', 'details_temp_table', 'details_temp_table', 'details_temp_table', 'details_temp_table', 'details_temp_table', 'header_temp_table']
Columns: ['id', 'insurer_id', 'first_name', 'middle_name', 'last_name', 'gender, marital_status', None, 'middle_name, first_name, last_name', None, None, None, 'address_type,address_line_1,address_line_2,state,zipcode,city', 'usage_type,phone', 'email', None, 'ssn', 'gender', 'marital_status', 'date_of_birth', 'date_of_birth', 'deceased_date', 'deceased_date,date_of_birth', 'deceased_date', 'spoken_language_1,spoken_language_2', 'company, job_role, deceased_date, date_of_birth, job_hiredate', 'rel

In [ ]:
import pandas as pd
attri = dict_mapping_attribute.values()

# Example dictionary
data = {'Source Table': tables,
        'Source Column': columns,
        'Attribute Name ': transformations_key,
        'transformation': transformations}

# Create DataFrame from dictionary
df = pd.DataFrame.from_dict(data)

# Print the DataFrame
print(df)


          Source Table                                      Source Column  \
0    header_temp_table                                                 id   
1    header_temp_table                                         insurer_id   
2   details_temp_table                                         first_name   
3   details_temp_table                                        middle_name   
4   details_temp_table                                          last_name   
5   details_temp_table                             gender, marital_status   
6                 None                                               None   
7   details_temp_table                 middle_name, first_name, last_name   
8                 None                                               None   
9                 None                                               None   
10                None                                               None   
11  address_temp_table  address_type,address_line_1,address_line_2,sta...   

In [ ]:
# Example dictionary
data = {'Source Table': tables,
        'Source Column': columns,
        'Attribute Name ': transformations_key,
        'transformation': transformations}

# Create DataFrame from dictionary
df = pd.DataFrame.from_dict(data)

# Print the DataFrame
df


,Source Table,Source Column,Attribute Name,transformation
0,header_temp_table,id,source_id,h.id
1,header_temp_table,insurer_id,subscriber_id,h.insurer_id
2,details_temp_table,first_name,first_name,d.first_name
3,details_temp_table,middle_name,middle_name,d.middle_name
4,details_temp_table,last_name,last_name,d.last_name
5,details_temp_table,"gender, marital_status",prefix_name,CASE\n WHEN d.gender = 'M' THEN 'Mr.'\n ...
6,None,None,suffix_name,NULL
7,details_temp_table,"middle_name, first_name, last_name",full_name,CASE\n WHEN d.middle_name IS NULL THEN CONC...
8,None,None,record_source,'Nova'
9,None,None,record_timestamp,CURRENT_TIMESTAMP()


In [ ]:
df.to_excel('required_outputs.xlsx', index=False)